## Bot Vs Bot 

* 测试game 版本速度

todo
* 测试 deep copy

In [1]:
# 环境配置
%cd ../
import sys
sys.path.append('./python')

d:\sgd-代码库\torch2.0-paly\sgd_deep_learning\sgd_rl\gobang


In [2]:
from dlgobang import GameState, Player
from dlgobang.agent import RandomBot
from dlgobang.utils import print_board, print_move
import time
import cProfile
import pstats

from IPython.display import clear_output

### 打印游戏对局
<1> We set a sleep timer to 0.3 seconds so that bot moves aren't printed too fast to observe   
<2> Before each move we clear the screen. This way the board is always printed to the same position on the command line.

In [3]:
def main():
    board_size = 9
    game = GameState.new_game(board_size)
    bots = {
        Player.BLACK: RandomBot(),
        Player.WHITE: RandomBot(),
    }
    print_board(game.board)

    while not game.is_over():
        # time.sleep(0.1)  # <1>
        # clear_output(wait=True) # 显示有点bug，不一定每次都能打印出结果
        
        bot_move = bots[game.next_player].select_move(game)
        print_move(game.next_player, bot_move)
        game = game.apply_move(bot_move)
        print_board(game.board)

    print(game.winner(), "win")

# 运行性能分析并保存到文件
cProfile.run('main()', 'profile_game')

# 读取分析结果并按时间排序
with open("profile_stats_game.txt", "w") as f:
    p = pstats.Stats('profile_game', stream=f)
    p.sort_stats('time').print_stats()

 9  .  .  .  .  .  .  .  .  . 
 8  .  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  .  . 
    A  B  C  D  E  F  G  H  J
Player.BLACK E9
 9  .  .  .  .  x  .  .  .  . 
 8  .  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  .  . 
    A  B  C  D  E  F  G  H  J
Player.WHITE A5
 9  .  .  .  .  x  .  .  .  . 
 8  .  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  .  . 
 5  o  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  .  . 
    A  B  C  D  E  F  G  H  J
Player.BLACK J2
 9  .  .  .  .  x  .  .  

### 对局速度测试

    优化后
    五子棋单局博弈速度
    10 rounds: avg time:0.04s

    [done]
    1、添加游戏状态的deepcopy，需要注意复制带来的额外开销 (!!todo deepcopy frozenset 那套机制没完全搞懂)
    2、以及游戏判定结束需要的时间（稍微有点丑陋的实现）
    
    [todo]
    3、直接用最简单的添加方式，并用复杂的判定条件，不知道是不是也很快。（直观解法:待测试，不是很急）

In [4]:
def oneround_time(game, bots):    
    ts = time.time()

    while not game.is_over():
        bot_move = bots[game.next_player].select_move(game)
        # print_move(game.next_player, bot_move)
        game = game.apply_move(bot_move)
    
    te = time.time()
    print("one rounds time:{:.2f}s".format(te-ts))
    return te-ts

def test_game_speed(game, n=2):
    bots = {
        Player.BLACK: RandomBot(),
        Player.WHITE: RandomBot(),
    }

    total_time = 0
    for _ in range(n):
        total_time += oneround_time(game, bots)
    
    avg_time = total_time/n
    print("{} rounds: avg time:{:.2f}s".format(n, avg_time))

In [5]:
from dlgobang.game import GameState, Player

board_size = 9

game = GameState.new_game(board_size)

print("五子棋单局博弈速度")
test_game_speed(game, n=10)

# 启动后就很快，第一局创建对象就特别慢


五子棋单局博弈速度
one rounds time:0.02s
one rounds time:0.02s
one rounds time:0.02s
one rounds time:0.02s
one rounds time:0.05s
one rounds time:0.05s
one rounds time:0.02s
one rounds time:0.04s
one rounds time:0.04s
one rounds time:0.04s
10 rounds: avg time:0.03s


## 测试 deep copy
    move 前后两个gamestate， line的变化，linemanager的变化
    后者是否会影响到前者